In [2]:
#Thanks a lot to: 
#1. Statsbomb/FBref for publicly sharing stats 
#2. Shreyas Khatri (Twitter: @khatri_shreyas) whose piece of code I used  to download  files from FBRef.
#OG link: https://github.com/shreyas7kha/FootballFbrefPlotting

import os
import numpy as np
import pandas as pd
import requests
import seaborn as sns
from bs4 import BeautifulSoup as soup

from datetime import date

path = "E:/2nd year/3rd sem/Data science/Project/Players data"    
os.chdir(path)
    
# Returns the current local date
today = date.today()

path = os.getcwd() + '/FBREF Files for ' + str(today)
os.mkdir(path)

# A function which can read data from a Fbref webpage
def readfromhtml(filepath):
    df = pd.read_html(filepath)[0]
    
    column_lst = list(df.columns)
    unique_col_names = []
    for col in column_lst:
        if col[1] not in unique_col_names:
            unique_col_names.append(col[1])
        else:
            unique_col_names.append(col[0]+' '+col[1])

    df.columns = unique_col_names
    df.drop(df[df['Player'] == 'Player'].index, inplace=True)
    df = df.fillna('0')
    df.set_index('Rk', drop=True, inplace=True)
    try:
        df['Comp'] = df['Comp'].apply(lambda x: ' '.join(x.split()[1:]))
        df['Nation'] = df['Nation'].astype(str)
        df['Nation'] = df['Nation'].apply(lambda x: x.split()[-1])
    except:
        print('Error in uploading file:' + filepath)
    finally:
        df = df.apply(pd.to_numeric, errors='ignore')
        return df


# If you want all data for the big 5 leagues, you just need to run this function with
# the filepath where you want to save all the files
# def save_all_csvs(base_url='https://fbref.com/en/comps/Big5/Big-5-European-Leagues-Stats',
#                   filepath=os.getcwd()):

def save_all_csvs(base_url='https://fbref.com/en/comps/Big5/2022-2023/2022-2023-Big-5-European-Leagues-Stats',
                  filepath=path):

    req = requests.get(base_url)
    parse_soup = soup(req.content, 'lxml')
    scripts = parse_soup.find_all('ul')
    url_list = scripts[4]
    urls = []
    for url in url_list.find_all('a', href=True):
        urls.append(url['href'])
    urls = [base_url[:17] + url for url in urls]
    for url in urls:
        df = readfromhtml(url)
        filename = url.split('/')[7]
        try:
            df.to_csv(filepath + '//' + filename + '.csv', encoding='utf-8-sig')
        except:
            print('An error occurred in saving the file')
        else:
            print('File has been saved as {0} at {1} in format YYYY-MM-DD '.format(filename, filepath))

In [3]:
save_all_csvs()

File has been saved as stats at E:\2nd year\3rd sem\Data science\Project\Players data/FBREF Files for 2023-09-15 in format YYYY-MM-DD 
File has been saved as keepers at E:\2nd year\3rd sem\Data science\Project\Players data/FBREF Files for 2023-09-15 in format YYYY-MM-DD 
File has been saved as keepersadv at E:\2nd year\3rd sem\Data science\Project\Players data/FBREF Files for 2023-09-15 in format YYYY-MM-DD 
File has been saved as shooting at E:\2nd year\3rd sem\Data science\Project\Players data/FBREF Files for 2023-09-15 in format YYYY-MM-DD 
File has been saved as passing at E:\2nd year\3rd sem\Data science\Project\Players data/FBREF Files for 2023-09-15 in format YYYY-MM-DD 
File has been saved as passing_types at E:\2nd year\3rd sem\Data science\Project\Players data/FBREF Files for 2023-09-15 in format YYYY-MM-DD 
File has been saved as gca at E:\2nd year\3rd sem\Data science\Project\Players data/FBREF Files for 2023-09-15 in format YYYY-MM-DD 
File has been saved as defense at E:\

In [5]:
possesion = pd.read_csv('E:/2nd year/3rd sem/Data science/Project/Players data/FBREF Files for 2023-09-15/possession.csv')
playingtime = pd.read_csv('E:/2nd year/3rd sem/Data science/Project/Players data/FBREF Files for 2023-09-15/playingtime.csv')
shooting =pd.read_csv('E:/2nd year/3rd sem/Data science/Project/Players data/FBREF Files for 2023-09-15/shooting.csv')
keepersadv =pd.read_csv('E:/2nd year/3rd sem/Data science/Project/Players data/FBREF Files for 2023-09-15/keepersadv.csv')
keepers =pd.read_csv('E:/2nd year/3rd sem/Data science/Project/Players data/FBREF Files for 2023-09-15/keepers.csv')
passing =pd.read_csv('E:/2nd year/3rd sem/Data science/Project/Players data/FBREF Files for 2023-09-15/passing.csv')
misc =pd.read_csv('E:/2nd year/3rd sem/Data science/Project/Players data/FBREF Files for 2023-09-15/misc.csv')
gca =pd.read_csv('E:/2nd year/3rd sem/Data science/Project/Players data/FBREF Files for 2023-09-15/gca.csv')
defense =pd.read_csv('E:/2nd year/3rd sem/Data science/Project/Players data/FBREF Files for 2023-09-15/defense.csv')
passes=pd.read_csv('E:/2nd year/3rd sem/Data science/Project/Players data/FBREF Files for 2023-09-15/passing_types.csv')
stats=pd.read_csv('E:/2nd year/3rd sem/Data science/Project/Players data/FBREF Files for 2023-09-15/stats.csv')

In [7]:


# Assuming you have a list of DataFrames: possesion, playingtime, shooting, passing, misc, gca, defense, passes, stats
dfs = [possesion, playingtime, shooting, passing, misc, gca, defense, passes, stats]

# Join DataFrames in the list using pd.concat()
df1 = pd.merge(pd.merge(possesion,playingtime,on='Rk'),shooting,on='Rk')
df1 = df1.T.drop_duplicates().T
df2 = pd.merge(pd.merge(passing,misc,on='Rk'),gca,on='Rk')
df2 = df2.T.drop_duplicates().T
df3= pd.merge(pd.merge(passes,defense,on='Rk'),stats,on='Rk')
df3 = df3.T.drop_duplicates().T
df_outfield2022_2023 = pd.merge(pd.merge(df1,df2,on='Rk'),df3,on='Rk')

# Remove duplicate columns
df_outfield2022_2023 = df_outfield2022_2023.T.drop_duplicates().T

# Save the merged DataFrame as a CSV file
df_outfield2022_2023.to_csv('E:/2nd year/3rd sem/Data science/Project/Players data/All_outfield.csv', index=False)

# Display the resulting DataFrame
df_outfield2022_2023


,Rk,Player_x_x,Nation_x_x,Pos_x_x,Squad_x_x,Comp_x_x,Age_x_x,Born_x_x,90s_x_x,Touches,...,Per 90 Minutes Gls,Per 90 Minutes Ast,Per 90 Minutes G+A,Per 90 Minutes G-PK,G+A-PK,Per 90 Minutes xG,Per 90 Minutes xAG,xG+xAG,Per 90 Minutes npxG,Per 90 Minutes npxG+xAG
0,1,Brenden Aaronson,USA,"MF,FW",Leeds United,Premier League,21,2000,26.4,1143,...,0.04,0.11,0.15,0.04,0.15,0.15,0.16,0.31,0.15,0.31
1,2,Paxten Aaronson,USA,"MF,DF",Eint Frankfurt,Bundesliga,18,2003,1.9,99,...,0.0,0.0,0.0,0.0,0.0,0.09,0.03,0.11,0.09,0.11
2,3,James Abankwah,IRL,DF,Udinese,Serie A,18,2004,0.7,39,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,George Abbott,ENG,MF,Tottenham,Premier League,16,2005,0.0,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Yunis Abdelhamid,MAR,DF,Reims,Ligue 1,34,1987,37.0,2459,...,0.03,0.05,0.08,0.03,0.08,0.06,0.03,0.09,0.06,0.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2884,2885,Martin Ødegaard,NOR,MF,Arsenal,Premier League,23,1998,34.7,2100,...,0.43,0.2,0.63,0.43,0.63,0.29,0.23,0.52,0.29,0.52
2885,2886,Milan Đurić,BIH,FW,Hellas Verona,Serie A,32,1990,15.2,644,...,0.07,0.07,0.13,0.07,0.13,0.09,0.08,0.18,0.09,0.18
2886,2887,Filip Đuričić,SRB,"MF,FW",Sampdoria,Serie A,30,1992,24.1,992,...,0.12,0.0,0.12,0.12,0.12,0.11,0.08,0.19,0.11,0.19
2887,2888,Blanco,0,MF,Cádiz,La Liga,22,2000,1.8,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
df_allkeeper2022_2023 = pd.merge(keepers,keepersadv)
df_allkeeper2022_2023.T.drop_duplicates().T
df_allkeeper2022_2023.to_csv('E:/2nd year/3rd sem/Data science/Project/Players data/All_keeper.csv',index=False)

df_allkeeper2022_2023

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,MP,Starts,...,AvgLen,Goal Kicks Att,Goal Kicks Launch%,Goal Kicks AvgLen,Opp,Stp,Stp%,#OPA,#OPA/90,AvgDist
0,1,Álvaro Aceves,ESP,GK,Valladolid,La Liga,19,2003,1,0,...,25.7,1,0.0,24.0,1,0,0.0,2,13.85,33.0
1,2,Julen Agirrezabala,ESP,GK,Athletic Club,La Liga,21,2000,8,7,...,30.5,62,77.4,55.4,96,11,11.5,10,1.33,15.4
2,3,Doğan Alemdar,TUR,GK,Rennes,Ligue 1,19,2002,6,4,...,33.0,35,71.4,51.4,75,5,6.7,5,1.11,14.8
3,4,Alisson,BRA,GK,Liverpool,Premier League,29,1992,37,37,...,26.9,193,26.9,31.3,397,23,5.8,89,2.41,19.8
4,5,Alphonse Areola,FRA,GK,West Ham,Premier League,29,1993,5,2,...,33.4,30,43.3,33.0,47,2,4.3,1,0.29,10.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,204,Joseph Whitworth,ENG,GK,Crystal Palace,Premier League,18,2004,2,2,...,30.1,15,46.7,36.7,25,1,4.0,3,1.50,15.4
204,205,Robin Zentner,GER,GK,Mainz 05,Bundesliga,27,1994,26,26,...,40.0,194,71.1,54.1,321,43,13.4,53,2.04,17.4
205,206,Michael Zetterer,GER,GK,Werder Bremen,Bundesliga,27,1995,2,1,...,41.6,12,66.7,50.3,15,3,20.0,4,2.37,16.5
206,207,Jeroen Zoet,NED,GK,Spezia,Serie A,31,1991,5,4,...,39.0,23,78.3,58.4,62,4,6.5,6,1.61,14.7
